In [1]:
import pandas as pd

# Charger le fichier CSV
file_path = 'Engineering_graduate_salary.csv'
data = pd.read_csv(file_path)

# Afficher les premières lignes pour avoir un aperçu des données
print(data.head())

# Obtenir les informations sur les types de données et les valeurs manquantes
print(data.info())

# Calculer et afficher les statistiques descriptives pour les colonnes numériques
print(data.describe())


       ID Gender         DOB  10percentage                       10board  \
0  604399      f  1990-10-22         87.80                          cbse   
1  988334      m  1990-05-15         57.00                          cbse   
2  301647      m  1989-08-21         77.33  maharashtra state board,pune   
3  582313      m  1991-05-04         84.30                          cbse   
4  339001      f  1990-10-30         82.00                          cbse   

   12graduation  12percentage                    12board  CollegeID  \
0          2009         84.00                       cbse       6920   
1          2010         64.50                       cbse       6624   
2          2007         85.17  amravati divisional board       9084   
3          2009         86.00                       cbse       8195   
4          2008         75.00                       cbse       4889   

   CollegeTier  ... MechanicalEngg ElectricalEngg  TelecomEngg  CivilEngg  \
0            1  ...             -1     

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# Charger les données
data = pd.read_csv('Engineering_graduate_salary.csv')

# Sélectionner les caractéristiques et la cible
X = data.drop(columns=['ID', 'CollegeCityID', 'CollegeID', 'Salary'])  # Exclure l'ID et la cible (Salary) de l'ensemble des caractéristiques
y = data['Salary']  # La cible est le salaire

# Diviser en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identifier les types de colonnes
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X_train.select_dtypes(include=['object']).columns

# Créer les pipelines pour les transformations numériques et catégorielles
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Imputation des valeurs manquantes par la moyenne
    ('scaler', StandardScaler())  # Normalisation des caractéristiques numériques
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputation des valeurs manquantes par la valeur la plus fréquente
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Encodage One-Hot pour les caractéristiques catégorielles
])

# Préprocesseur à appliquer sur les colonnes
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Appliquer le prétraitement sur les ensembles d'entraînement et de test
X_train_prepared = preprocessor.fit_transform(X_train)
X_test_prepared = preprocessor.transform(X_test)

# Le modèle peut ensuite être construit et entraîné avec ces données préparées.


In [3]:
from keras.models import Sequential
from keras.layers import Dense

# Définir l'architecture du modèle
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(X_train_prepared.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='linear'))  # Couche de sortie pour la régression

# Compiler le modèle
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

# Entraîner le modèle
model.fit(X_train_prepared, y_train, epochs=100, validation_split=0.2)

# Évaluer le modèle
model.evaluate(X_test_prepared, y_test)


Epoch 1/100


C:\Users\zaebo\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 144386572288.0000 - mean_absolute_error: 308331.9062 - val_loss: 104615329792.0000 - val_mean_absolute_error: 287366.5000
Epoch 2/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 157724033024.0000 - mean_absolute_error: 308496.9375 - val_loss: 98698682368.0000 - val_mean_absolute_error: 277690.2812
Epoch 3/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 176042000384.0000 - mean_absolute_error: 305797.4688 - val_loss: 68850925568.0000 - val_mean_absolute_error: 222631.9688
Epoch 4/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 126773436416.0000 - mean_absolute_error: 227658.3281 - val_loss: 24875360256.0000 - val_mean_absolute_error: 111280.0078
Epoch 5/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 40859238400.0000 - mean_absolute_error: 116908.6172 - val_loss: 18036936704.0000 - val_mean_absolute_error: 98709.5703
Epoch 6/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 40763113472.0000 - mean_absolute_error: 113320.4453 - 

[60395274240.0, 170430.59375]

In [4]:
#!pip install -U keras-tuner
from kerastuner.tuners import RandomSearch

def build_model(hp):
    model = Sequential()
    model.add(Dense(hp.Int('neurons', min_value=20, max_value=300, step=10), activation='relu', input_dim=X_train_prepared.shape[1]))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='helloworld')

tuner.search(X_train_prepared, y_train, epochs=10, validation_data=(X_test_prepared, y_test))


Reloading Tuner from my_dir\helloworld\tuner0.json


C:\Users\zaebo\AppData\Local\Temp\ipykernel_27356\73266157.py:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [5]:
# Obtenir les meilleurs hyperparamètres
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Afficher les meilleurs hyperparamètres
print(f"Meilleurs hyperparamètres: {best_hps.values}")


Meilleurs hyperparamètres: {'neurons': 100}
